In [1]:
#try xgboost
import pandas as pd
import numpy as np
import xgboost as xgb

In [2]:
df = pd.read_csv('/Users/frankieshen/Desktop/ooschool/專案/stock_0050_final.csv', parse_dates = True, index_col = 0)

In [3]:
print(df.columns)
print(df.shape)
print(type(df))

Index(['Unnamed: 0', 'Capacity', 'Volume', 'Open', 'High', 'Low', 'Close',
       'Change', 'Transcation', 'Tomorrow', 'Target', 'SMA_20', 'RSI', 'slowk',
       'slowd', 'MACD', 'MACD_signal', 'MACD_hist', 'upper_band',
       'middle_band', 'lower_band'],
      dtype='object')
(3483, 21)
<class 'pandas.core.frame.DataFrame'>


In [4]:
df.head()

,Unnamed: 0,Capacity,Volume,Open,High,Low,Close,Change,Transcation,Tomorrow,...,SMA_20,RSI,slowk,slowd,MACD,MACD_signal,MACD_hist,upper_band,middle_band,lower_band
Date,,,,,,,,,,,,,,,,,,,,,
2010-02-26,33,9391284,483837797,51.65,51.75,51.20,51.55,-0.10,1366,52.80,...,52.4075,38.424149,46.288720,50.743915,-1.500417,-1.927544,0.427127,54.411814,52.4075,50.403186
2010-03-01,34,13647130,715419506,51.50,52.95,51.50,52.80,1.25,2066,52.65,...,52.3275,47.850461,58.861819,52.243188,-1.336577,-1.809351,0.472774,54.124296,52.3275,50.530704
2010-03-02,35,9997438,528112268,52.75,53.10,52.55,52.65,-0.15,1225,52.65,...,52.2500,46.922188,71.092962,58.747834,-1.204946,-1.688470,0.483524,53.838710,52.2500,50.661290
2010-03-03,36,8764746,462426932,52.70,53.10,52.50,52.65,0.00,1041,52.20,...,52.2350,46.922188,84.210526,71.388436,-1.088085,-1.568393,0.480308,53.802514,52.2350,50.667486
2010-03-04,37,8149453,426373974,52.75,52.90,51.80,52.20,-0.45,1295,52.95,...,52.2000,43.955363,77.192982,77.498824,-1.020025,-1.458719,0.438694,53.737530,52.2000,50.662470


In [5]:
df_reorder = df[['Volume','Open', 'High', 'Low', 
              'SMA_20', 'RSI', 'slowk', 'slowd', 'MACD', 'MACD_signal', 'MACD_hist', 'upper_band', 'middle_band', 'lower_band', 'Close']]

In [6]:
print(df_reorder.columns)
print(df_reorder.shape)
print(type(df_reorder))

Index(['Volume', 'Open', 'High', 'Low', 'SMA_20', 'RSI', 'slowk', 'slowd',
       'MACD', 'MACD_signal', 'MACD_hist', 'upper_band', 'middle_band',
       'lower_band', 'Close'],
      dtype='object')
(3483, 15)
<class 'pandas.core.frame.DataFrame'>


In [7]:

def windowing(train, val, WINDOW, PREDICTION_SCOPE):
    assert type(train) == np.ndarray, "train must be passed as an array"
    assert type(val) == np.ndarray, "validation must be passed as an array"
    assert type(WINDOW) == int, "Window must be an integer"
    assert type(PREDICTION_SCOPE) == int, "Prediction score must be an integer"

    x_train = []
    y_train = []
    x_test = []
    y_test = []

    for i in range(len(train) - (WINDOW + PREDICTION_SCOPE)):
        x, y = np.array(train[i:i + WINDOW, :-1]), np.array(train[i + WINDOW + PREDICTION_SCOPE, -1])
        x_train.append(x)
        y_train.append(y)
    for i in range(len(val) - (WINDOW + PREDICTION_SCOPE)):
        x, y = np.array(val[i:i + WINDOW, :-1]), np.array(val[i + WINDOW + PREDICTION_SCOPE, -1])
        x_test.append(x)
        y_test.append(y)


    return x_train, y_train, x_test, y_test

In [8]:
def train_test_split(data, percentage):
    assert type(data) == pd.core.frame.DataFrame, "data must be a dataframe"
    assert type(percentage) == float, "percentage must be a float"

    train = data.iloc[:int(len(data) * percentage)]
    test = data.iloc[int(len(data) * percentage):]
    
    return train, test

In [9]:
def train_validation_split(train, percentage):
    assert type(train) == pd.core.frame.DataFrame, "train musst be a dataframe"
    assert type(percentage) == float, "percentage musst be a float"
    
    train_set = np.array(train.iloc[:int(len(train) * percentage)])
    validation_set = np.array(train.iloc[int(len(train) * percentage):])
    
    return train_set, validation_set

In [10]:
percentage = 0.8
WINDOW = 5
PREDICTION_SCOPE = 0

In [11]:
train, test = train_test_split(df_reorder, percentage)
train_set, validation_set = train_validation_split(train, percentage)
print(f"train_set shape: {train_set.shape}")
print(f"validation_set shape: {validation_set.shape}")
print(f"test shape: {test.shape}")
print(f"train shape: {train.shape}")

train_set shape: (2228, 15)
validation_set shape: (558, 15)
test shape: (697, 15)
train shape: (2786, 15)


In [12]:
X_train, y_train, X_val, y_val = windowing(train_set, validation_set, WINDOW, PREDICTION_SCOPE)
#Convert the returned list into arrays
X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_val shape: {y_val.shape}")

X_train shape: (2223, 5, 14)
y_train shape: (2223,)
X_val shape: (553, 5, 14)
y_val shape: (553,)


In [13]:
#xgboost only allows two dimensional input
#reshape x_train and x_val into two dimension by joining the rows of the windowed days into one big input
X_train_2dim = X_train.reshape(X_train.shape[0], -1)
X_val_2dim = X_val.reshape(X_val.shape[0], -1)
print(f"X_train_new shape: {X_train_2dim.shape}")
print(f"X_val_new shape: {X_val_2dim.shape}")

X_train_new shape: (2223, 70)
X_val_new shape: (553, 70)


In [14]:
xgb_model = xgb.XGBRegressor(gamma=1, n_estimators=200)
xgb_model.fit(X_train_2dim, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=1, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [15]:
from sklearn.metrics import mean_absolute_error

In [16]:
pred_val = xgb_model.predict(X_val_2dim)
mae_val = mean_absolute_error(y_val, pred_val)
mae_val

16.271520689814256

In [26]:
#test the model
test_array = np.array(test)

In [28]:
test_array

array([[8.20657527e+08, 1.35600000e+02, 1.36050000e+02, ...,
        1.36685000e+02, 1.33561044e+02, 1.34850000e+02],
       [8.87399669e+08, 1.35000000e+02, 1.37150000e+02, ...,
        1.36817500e+02, 1.33872797e+02, 1.37050000e+02],
       [8.61039047e+08, 1.37050000e+02, 1.37250000e+02, ...,
        1.36927500e+02, 1.34179505e+02, 1.36700000e+02],
       ...,
       [1.28963079e+09, 1.57300000e+02, 1.59100000e+02, ...,
        1.57685000e+02, 1.50253237e+02, 1.58800000e+02],
       [7.61950562e+08, 1.58600000e+02, 1.59350000e+02, ...,
        1.57702500e+02, 1.50267147e+02, 1.58250000e+02],
       [1.04227977e+09, 1.57350000e+02, 1.57350000e+02, ...,
        1.57640000e+02, 1.50174573e+02, 1.56150000e+02]])

In [34]:
# test the model

X_test = []
y_test = []

for i in range(len(test_array) - (WINDOW + PREDICTION_SCOPE)):
    x, y = np.array(test_array[i:i + WINDOW, :-1]), np.array(test_array[i + WINDOW + PREDICTION_SCOPE, -1])
    X_test.append(x)
    y_test.append(y)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [39]:
print(test.shape)
print(X_test.shape)
print(y_test.shape)

(697, 15)
(692, 5, 14)
(692,)


In [43]:
#reshape X_test into 2 dimension array
X_test_2dim = X_test.reshape(X_test.shape[0], -1)
print(X_test_2dim)
print(X_test_2dim.shape)

[[8.20657527e+08 1.35600000e+02 1.36050000e+02 ... 1.39358453e+02
  1.37167500e+02 1.34976547e+02]
 [8.87399669e+08 1.35000000e+02 1.37150000e+02 ... 1.39391198e+02
  1.37192500e+02 1.34993802e+02]
 [8.61039047e+08 1.37050000e+02 1.37250000e+02 ... 1.39575539e+02
  1.37255000e+02 1.34934461e+02]
 ...
 [1.50595993e+09 1.50400000e+02 1.51750000e+02 ... 1.65010889e+02
  1.57590000e+02 1.50169111e+02]
 [8.68908321e+08 1.52150000e+02 1.52450000e+02 ... 1.65116763e+02
  1.57685000e+02 1.50253237e+02]
 [1.25853996e+09 1.54000000e+02 1.56500000e+02 ... 1.65137853e+02
  1.57702500e+02 1.50267147e+02]]
(692, 70)


In [44]:
xgb_model.fit(X_test_2dim, y_test)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=1, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [46]:
pred_test = xgb_model.predict(X_test_2dim)
mae_test = mean_absolute_error(y_test, pred_test)
mae_test

0.34766466879431246

In [50]:
X_test_2dim

array([[8.20657527e+08, 1.35600000e+02, 1.36050000e+02, ...,
        1.39358453e+02, 1.37167500e+02, 1.34976547e+02],
       [8.87399669e+08, 1.35000000e+02, 1.37150000e+02, ...,
        1.39391198e+02, 1.37192500e+02, 1.34993802e+02],
       [8.61039047e+08, 1.37050000e+02, 1.37250000e+02, ...,
        1.39575539e+02, 1.37255000e+02, 1.34934461e+02],
       ...,
       [1.50595993e+09, 1.50400000e+02, 1.51750000e+02, ...,
        1.65010889e+02, 1.57590000e+02, 1.50169111e+02],
       [8.68908321e+08, 1.52150000e+02, 1.52450000e+02, ...,
        1.65116763e+02, 1.57685000e+02, 1.50253237e+02],
       [1.25853996e+09, 1.54000000e+02, 1.56500000e+02, ...,
        1.65137853e+02, 1.57702500e+02, 1.50267147e+02]])

In [81]:
#投資回測分析
test_new = test.iloc[:-5]
test_new["prediction"] = pred_test

/var/folders/xf/rrndggj14ddg8j43w4kdc53m0000gn/T/ipykernel_17188/3619392212.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_new["prediction"] = pred_test


In [83]:
test_new["pred_tomorrow"] = test_new["prediction"].shift(-1)
test_new["target"] = (test_new["pred_tomorrow"] > test_new["prediction"]).astype(int)
test_new["status"] = test_new["target"].shift(1)
test_new

/var/folders/xf/rrndggj14ddg8j43w4kdc53m0000gn/T/ipykernel_17188/4248799802.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_new["pred_tomorrow"] = test_new["prediction"].shift(-1)
/var/folders/xf/rrndggj14ddg8j43w4kdc53m0000gn/T/ipykernel_17188/4248799802.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_new["target"] = (test_new["pred_tomorrow"] > test_new["prediction"]).astype(int)
/var/folders/xf/rrndggj14ddg8j43w4kdc53m0000gn/T/ipykernel_17188/4248799802.py:3: SettingWithCopyWarning: 
A 

,Volume,Open,High,Low,SMA_20,RSI,slowk,slowd,MACD,MACD_signal,MACD_hist,upper_band,middle_band,lower_band,Close,prediction,pred_tomorrow,target,status
Date,,,,,,,,,,,,,,,,,,,
2021-06-22,820657527,135.60,136.05,134.50,136.6850,45.090287,29.020666,56.562134,0.431355,0.625619,-0.194263,139.808956,136.6850,133.561044,134.85,137.550995,138.198807,1,NaN
2021-06-23,887399669,135.00,137.15,134.80,136.8175,52.600508,22.239057,35.352197,0.429867,0.586468,-0.156602,139.762203,136.8175,133.872797,137.05,138.198807,138.708969,1,1.0
2021-06-24,861039047,137.05,137.25,136.45,136.9275,51.396127,37.777778,29.679167,0.395881,0.548351,-0.152470,139.675495,136.9275,134.179505,136.70,138.708969,138.660843,0,1.0
2021-06-25,911180118,137.45,137.95,136.70,137.0975,52.237365,53.333333,37.783389,0.384686,0.515618,-0.130932,139.367847,137.0975,134.827153,136.95,138.660843,138.688858,1,0.0
2021-06-28,560920233,137.00,137.50,136.50,137.1675,53.111343,54.444444,48.518519,0.391474,0.490789,-0.099315,139.358453,137.1675,134.976547,137.20,138.688858,140.094894,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-18,1087753112,156.35,158.95,155.80,158.6625,56.894590,30.038760,37.238784,2.388459,3.221403,-0.832944,163.620511,158.6625,153.704489,158.95,153.793320,155.770309,1,0.0
2024-04-19,5120010612,154.10,154.15,149.50,158.5475,39.815879,29.919822,30.687597,1.568045,2.890731,-1.322686,164.080354,158.5475,153.014646,150.90,155.770309,157.729218,1,1.0
2024-04-22,1505959931,150.40,151.75,149.70,158.2000,39.109133,21.934906,27.297829,0.871504,2.486886,-1.615382,164.755913,158.2000,151.644087,150.45,157.729218,157.970169,1,1.0


In [84]:
test_new.to_csv("stock_0050_testset.csv", index = False)

In [ ]:
###從這邊開始
# 所以什麼時候要買股票就很好找了：status 從 0 變成 1 的時候，1 的那天的開盤買進（因為 status 已經位移一天了喔）
# 從 prediction 的角度解釋就是：當 A.I. 的預測從 0 變成 1 的時候，1 的隔天的開盤買進
test_new["buy_cost"] = test_new.Open[np.where((test_new.target == 1) * (test_new.target.shift(1) == 0))[0]]

# 同理，賣股票也很好找：status 從 1 變成 0 的時候，0 的那天的開盤賣出
test_new["sell_cost"] = test_new.Open[np.where((test_new.target == 0) * (test_new.target.shift(1) == 1))[0]]

test_new = test_new.fillna(0)

In [77]:
# 來算算每次買賣的報酬率吧！
# 一買一賣是剛好對應的，所以把買的成本以及賣的價格這兩欄的數字取出，就能輕易的算出交易報酬率
buy_cost = np.array(test_new[test_new.buy_cost != 0])
sell_price = np.array(test_new[test_new.sell_cost != 0])

In [ ]:
# 所以什麼時候要買股票就很好找了：status 從 0 變成 1 的時候，1 的那天的開盤買進（因為 status 已經位移一天了喔）
# 從 prediction 的角度解釋就是：當 A.I. 的預測從 0 變成 1 的時候，1 的隔天的開盤買進
test['buy_cost'] = test.open[np.where((test.status == 1) * (test.status.shift(1) == 0))[0]]
# 同理，賣股票也很好找：status 從 1 變成 0 的時候，0 的那天的開盤賣出
test['sell_cost'] = test.open[np.where((test.status == 0) * (test.status.shift(1) == 1))[0]]
# 把缺值補上 0
test = test.fillna(0)

# 來算算每次買賣的報酬率吧！
# 一買一賣是剛好對應的，所以把買的成本以及賣的價格這兩欄的數字取出，就能輕易的算出交易報酬率

buy_cost = np.array(test.buy_cost[test.buy_cost != 0])
sell_price = np.array(test.sell_cost[test.sell_cost != 0])

# 但是回測的最後一天，有時候會發現還有持股尚未賣出喔！由於還沒賣就不能當作一次完整的交易，
# 所以最後一次的買進，我們先忽略
if len(buy_cost) > len(sell_price) :
    buy_cost = buy_cost[:-1]

trade_return = sell_price / buy_cost - 1